**Disclaimer**

All codes in this calibration is taken either directly or heavily inspired by nicolainielsen32 on GitHub (linked as source in thesis and read me file). All computer vision codes are open source from the OpenCV library (also linked as source in thesis).

The codes are put in the repository to illustrate the process of developing a program, not to claim as the authors own work in its entirety.

Import packages

In [2]:
import cv2
import numpy as np

Main

In [1]:
def main():
    red_lower = np.array([0, 100, 122], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    green_lower = np.array([35, 20, 16], np.uint8)
    green_upper = np.array([110, 196, 197], np.uint8)

    
    paused = False
    index = 0

    vl_path = "Week16/left2.avi"
    vr_path = "Week16/right2.avi"
    
    vl = cv2.VideoCapture(vl_path)
    vr = cv2.VideoCapture(vr_path)

    while True:
        if not paused:
            retl, framel = vl.read()
            retr, framer = vr.read()
            if not retl or not retr:
                print("End of video")
                break

        #framer,framel = undistortRectify(framer,framel)
        hsvl = cv2.cvtColor(framel, cv2.COLOR_BGR2HSV)
        hsvr = cv2.cvtColor(framer,cv2.COLOR_BGR2HSV)

        mask_greenL = masker(hsvl, green_lower, green_upper)
        mask_redL = masker(hsvl, red_lower, red_upper)
        mask_greenR = masker(hsvr, green_lower, green_upper)
        mask_redR = masker(hsvr, red_lower, red_upper)

        contours_greenL, _ = cv2.findContours(mask_greenL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redL, _ = cv2.findContours(mask_redL.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersL = calculate_centers(contours_greenL)
        red_dot_centersL = calculate_centers(contours_redL)

        contours_greenR, _ = cv2.findContours(mask_greenR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_redR, _ = cv2.findContours(mask_redR.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        green_dot_centersR = calculate_centers(contours_greenR)
        red_dot_centersR = calculate_centers(contours_redR)

        for center in green_dot_centersL:
            cv2.circle(framel, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersL:
            cv2.circle(framel, center, 3, (255, 255, 255), -1)


        for center in green_dot_centersR:
            cv2.circle(framer, center, 3, (0, 255, 0), -1)
        for center in red_dot_centersR:
            cv2.circle(framer, center, 3, (255, 255, 255), -1)

        #depthsRed = calculate_depths(framer,framel,B,f_pixel,alpha,"Week16/Red/left","Week16/Red/right")
        #depthsGreen = calculate_depths(framer,framel,B,f_pixel,alpha,"Week16/Green/left","Week16/Green/right")
        
        framel,framer, depthRed = displaydepth(framel,framer,red_dot_centersL,red_dot_centersR,B,f_pixel,alpha)
        framel,framer, depthGreen = displaydepth(framel,framer,green_dot_centersL,green_dot_centersR,B,f_pixel,alpha)

        savecoords(red_dot_centersL,"Week22/RedAllLeft.txt",index)
        savecoords(red_dot_centersR,"Week22/RedAllRight.txt",index)
        savecoords(green_dot_centersL,"Week22/GreenAllLeft.txt",index)
        savecoords(green_dot_centersR,"Week22/GreenAllRight.txt",index)
        
        index += 1

        key = cv2.waitKey(1) & 0xFF
        if key == ord("p"):
            paused = not paused
            print("Paused" if paused else "Resumed")
        elif key == ord("d"):
            break


    vl.release()
    vr.release()

    cv2.destroyAllWindows()


Run main

In [ ]:
#Stereo vision setup parameters
frame_rate = 30     #Camera frame rate
B = 2.7             #Distance between the cameras [cm]
f = 3.67            #Camera lense's focal length [mm]
alpha = 78          #Camera field of view in the horisontal plane [degrees]
f_pixel = f * (1080/((1/2.88)*25.4))
#Camera focal length is 3.67mm for 920 pro and 78 degrees camera field of view
#For c930 it is approximately the same and 90 degrees camera field of view

if __name__ == "__main__":
    main()